In [1]:
import math

def tan_deg( tan ):
    tan_in_degree = math.radians(tan)
    return math.tan(tan_in_degree)

def sin_deg( sin ):
    sin_in_degree = math.radians(sin)
    return math.sin(sin_in_degree)

def cos_deg( cos ):
    cos_in_degree = math.radians(cos)
    return math.cos(cos_in_degree)

def vec_add (vec1 , vec2):
    a0 = vec1[0] + vec2[0]
    a1 = vec1[1] + vec2[1]
    a2 = vec1[2] + vec2[2]
    return (a0 , a1 , a2)

def polar_to_cartesian( r , theta , yaw = 0):
    y = r * (- sin_deg(yaw)) ;
    x = r * cos_deg(theta) * (cos_deg(yaw))
    z = r * sin_deg(theta)
    return (x , y , z)

def find_horizontal_angle( vec ):
    # in format (x , y , z)
    if( vec[1] != 0):
        return math.degrees(math.atan(vec[0] / vec[1]))
    else:
        return 0

def find_vertical_angle( vec ):
    # in format (x , y , z)
    if( vec[2] != 0):
        return math.degrees(math.atan(vec[2] / vec[1]))
    else:
        return 0

# the distance of turbine to the laser.
#turbine_dist = 10
#turbine_height = 100
#prop_length = 0.132
prop_length = 0.068
#vectors in form of (x , y , z)
turbine_vec = (-0.072216 , 2.818920 , 0.000000)
#turbine_height_vec = (0,0 , 1)
turbine_height_vec = (0.000000 , 0.000000 , 0.337754)
step = 3 # step on the circle in degrees
velocity = step / 6.0; #change rpm to equal times of sections
# Vector pointing to the center of the circle
center = vec_add(turbine_vec, turbine_height_vec)

points_matrix = []

for i in range(0, 360, step):
    print(i)
    vec_relative_to_center = polar_to_cartesian(prop_length , i , yaw = 10)#
    translation_vec = vec_add(vec_relative_to_center , center)#
    points_matrix.append(translation_vec)


vertical_angles = []
horizontal_angles = []

for index, i in enumerate(points_matrix):
    vertical_angles.append(find_vertical_angle(i))#
    horizontal_angles.append(find_horizontal_angle(i))#

vertical_angles_relative = []
horizontal_angles_relative = []

for i,j in enumerate(points_matrix):
    if ( i == 0) :
        vertical_angles_relative.append(vertical_angles[i])
        horizontal_angles_relative.append(horizontal_angles[i])
    else:
        vertical_angles_relative.append(vertical_angles[i] - vertical_angles[i - 1])
        horizontal_angles_relative.append(horizontal_angles[i] - horizontal_angles[i - 1])

#############################    write the code into file   ##########################################
# file = open("testfile.txt","w")
# firstH = 0 
# firstV = 0
# file.write("PROGRAM\n")
# file.write("\nDIM var1 AS DOUBLE\nDIM vel AS DOUBLE\n")
# file.write("\nvel = 1\n")
# file.write("\nABS\n")
# for i,j in enumerate(points_matrix):
#    if ( i == 0):
#        file.write("\nMOVEABS A %f B %f\n" % (horizontal_angles_relative[i] , vertical_angles_relative[i]))
#        file.write("\nWHILE var1 < 10\n\n")
#        firstH = horizontal_angles_relative[i]
#        firstV = vertical_angles_relative[i]
#    else:
#        #file.write("IF a== 2 THEN\n")
#        file.write("PVT A %f, 0 B %f, 0 TIME vel \n" %(horizontal_angles[i] , vertical_angles[i] ))
#        #file.write("END IF\n")
# file.write("PVT A %f, 0 B %f, 0 TIME vel \n" %(firstH , firstV ))
# file.write("\nvar1 = var1 + 1\n")
# file.write("\nWEND\n")
# file.write("\nEND PROGRAM")
# file.close()

#############################    write arrays into code         ######################################
file = open("testfile.txt","w")
firstH = 0 
firstV = 0 
file.write("PROGRAM\n\n")
file.write("DIM times AS DOUBLE\n")
file.write("DIM timesMax AS DOUBLE\n")
file.write("DIM vel AS DOUBLE\n")
file.write("DIM var1 AS DOUBLE\n\n")
file.write("DIM DoubleA() AS DOUBLE = {")
for i,j in enumerate(points_matrix): 
   if ( i == 0):
       #file.write("MOVEABS A %f B %f\n" % (horizontal_angles_relative[i] , vertical_angles_relative[i]))
       firstH = horizontal_angles_relative[i]
       firstV = vertical_angles_relative[i]
   else:
       #file.write("IF a== 2 THEN\n")
       file.write("%f" %(horizontal_angles[i]))
       #file.write("END IF\n")
       if(i != len(points_matrix)-1):
           file.write(",")
       else: 
           file.write(",%f}\n" %(firstH))

file.write("DIM DoubleB() AS DOUBLE = {")
for i,j in enumerate(points_matrix):
   if ( i == 0):
       #file.write("MOVEABS A %f B %f\n" % (horizontal_angles_relative[i] , vertical_angles_relative[i]))
       firstH = horizontal_angles_relative[i]
       firstV = vertical_angles_relative[i]
   else:
       #file.write("IF a== 2 THEN\n")
       file.write("%f" %(vertical_angles[i] ))
       #file.write("END IF\n")
       if (i != len(points_matrix)-1):
           file.write(",")
       else: 
           file.write(",%f}\n\n" %(firstV))

file.write("vel = 1 \n")
file.write("times = 0 \n")
file.write("timesMax = %d\n\n" %(len(points_matrix)))
file.write("ABS\n\n")
file.write("MOVEABS A %f B %f\n\n" % (firstH , firstV))
#file.write("WHILE var1 < 10\n")
file.write("WHILE times < timesMax\n")
file.write("   PVT A DoubleA(times), 0 B DoubleB(times), 0 TIME vel\n")
file.write("   times = times + 1\n")
file.write("   IF var1 < 10 THEN\n")
file.write("      IF times > timesMax - 1 THEN\n")
file.write("         times = 0\n")
file.write("         var1 = var1 + 1\n")
file.write("      END IF\n")
file.write("   END IF\n")
file.write("WEND\n\n")
#file.write("var1 = var1 + 1\n")
#file.write("WEND\n\n")
file.write("END PROGRAM")
file.close()


0
3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
48
51
54
57
60
63
66
69
72
75
78
81
84
87
90
93
96
99
102
105
108
111
114
117
120
123
126
129
132
135
138
141
144
147
150
153
156
159
162
165
168
171
174
177
180
183
186
189
192
195
198
201
204
207
210
213
216
219
222
225
228
231
234
237
240
243
246
249
252
255
258
261
264
267
270
273
276
279
282
285
288
291
294
297
300
303
306
309
312
315
318
321
324
327
330
333
336
339
342
345
348
351
354
357
